In [1]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import math
import controls as ctrl
import pandas as pd
import os

**Programmer:** Marco Gutierrez

# Project Structure

1. Stablish the constants for the model (endowments)
1. Define the payoff functions
1. Define a calibrator function for the tax rate (which gets the value that maximizes the total payoff)
1. Define a tester function which uses the calibrator for several sets of parameters
1. Use the tester to obtain the set of params which maximizes the difference in optimal tax rates

# 1. Defining our constants (endowments)

In [2]:
original_task_endowments = ctrl.task_endowments
total_endowment = sum(original_task_endowments)
unique_task_endowments = list(set(ctrl.task_endowments))
unique_task_endowments.sort()

In [4]:
original_task_endowments

[125, 80, 40, 25, 25, 15, 15, 15, 9]

In [3]:
total_endowment*1

349

# 2. Defining the Payoff Functions

## 1. Public Payoff

### Parameters

In [4]:
theta = 0.025 # exponential parameter

### Payoff Function

In [5]:
def public_payoff(total_endowment, tax_rate):
    """
    This function calculates the public payoff for the players using
    the Kamei (2019) functions
    
    Input: Theta (exponential parameter), the sum of all the endowments (total_endowment) and the Tax rate (tax_rate)
    Output: Public payoff for each player
    """
    total_public_contribution = tax_rate*total_endowment
    
    if total_public_contribution <= 192:
        # public income for current total contribution
        return (101 / (1 + 100 * math.exp(-theta * total_public_contribution)) - 1)
    else:
        return (101 / (1 + 100 * math.exp(-theta * 192)) - 1)

## 2. Private Payoff

### Parameters

In [6]:
gamma = 0.85 # Degree of private payoff function
alpha = ctrl.alpha # Private productivity additive parameter
beta = ctrl.beta # Private productivity multiplicative parameter

### Private Productivity Function

In [7]:
def private_productivity(alpha, beta, tax_rate, total_endowment):
    """
    Determines the productivity of private payoffs.
    
    Input: Function base parameters (alpha, beta), the current tax rate and the sum of all the endowments (total_endowment)
    Output: Private productivity
    """
    total_public_contribution = tax_rate*total_endowment
    
    if total_public_contribution <= 192:
        return ctrl.alpha + ctrl.beta*total_public_contribution
    else:
        return ctrl.alpha + ctrl.beta*192

### Payoff Function

In [8]:
def private_payoff(private_prod, gamma, tax_rate, endowment):
    """
    Determines the private payoff for each player.
    
    Input: Private productivity (value, not the funct), degree of private payoff function (gamma), 
           current tax rate and endowment
    Output: Private payoff
    """
    private_budget = (1-tax_rate)*endowment
    
    return (private_prod*private_budget)**gamma

# 3. Defining the tax rate calibrator

In [9]:
def total_payoff_maximizer(gamma, alpha, beta, endowment, total_endowment, tax_step):
    """
    This function tries several values for the tax rate in the total payoff
    and gives as a result the value that maximizes it for a player 
    of an specific base endowment
    
    Input: Private/Public payoff function parameters (gamma, alpha, beta), 
           player endowment, tax rate constant increase for calculations (0 <= tax_step <= 1)
    Output: List with numerically optimal tax rate for a player of an specific endowment (first item)
            and the optimal total payoff (second item)
    """
    
    optimal_tax_rate = 0 # float, value of optimal tax rate
    optimal_total_payoff = 0 # float, value of optimal total payoff
    
    for tax_rate in list(np.arange(0, 1+tax_step, tax_step)): # evaluating each tax rate
        player_public_payoff = public_payoff(total_endowment, tax_rate)
        player_private_productivity = private_productivity(alpha, beta, tax_rate, total_endowment)
        player_private_payoff = private_payoff(player_private_productivity, gamma, tax_rate, endowment)
        
        player_total_payoff = player_public_payoff + player_private_payoff
        
        if optimal_total_payoff < player_total_payoff:
            optimal_tax_rate = tax_rate
            optimal_total_payoff = player_total_payoff
        
    return [optimal_tax_rate, optimal_total_payoff]

### Testing our maximizer

In [16]:
optimal_set = total_payoff_maximizer(gamma, alpha, beta, unique_task_endowments[0], total_endowment, 0.005)
optimal_set

[0.545, 767.9794641897415]

# 4. Defining our tester

In [30]:
def tester_calibration(initial_gamma, initial_alpha, initial_beta, step_gamma, step_alpha, step_beta, step_tax):
    """
    Obtains the optimal tax rates for several set of parameters
    given the steps for each of them and calculates the difference
    of the ones for the higher and lower endowment players
    
    Input: Initial values for payoff function parameters
    Output: Dataframe with differences in taxes
    """
    
    # list of used parameters
    gamma_list = []
    beta_list = []
    alpha_list = []
    high_endw_tax_list = []
    low_endw_tax_list = []
    tax_differences = []
    
    for gamma in np.arange(initial_gamma, 2, step_gamma):
        for beta in np.arange(initial_beta, initial_beta+1, step_beta):
            for alpha in np.arange(initial_alpha, initial_alpha*50, step_alpha):
                high_endw_tax = total_payoff_maximizer(gamma, alpha, beta, unique_task_endowments[0], total_endowment, step_tax)[0]
                low_endw_tax = total_payoff_maximizer(gamma, alpha, beta, unique_task_endowments[1], total_endowment, step_tax)[0]
                
                tax_difference = high_endw_tax - low_endw_tax
                
                gamma_list.append(gamma)
                beta_list.append(beta)
                alpha_list.append(alpha)
                
                high_endw_tax_list.append(high_endw_tax)
                low_endw_tax_list.append(low_endw_tax)
                tax_differences.append(tax_difference)
    
    data = {"Gamma": gamma_list, 
            "Beta": beta_list, 
            "Alpha": alpha_list,
            "High Endowment Tax": high_endw_tax_list, 
            "Low Endowment Tax": low_endw_tax_list, 
            "Tax Difference": tax_differences}
    
    return pd.DataFrame(data, columns= ["Gamma", "Beta", "Alpha", "High Endowment Tax", "Low Endowment Tax","Tax Difference"])

# 5. Starting the Calibration

### Initial Parameters

In [28]:
gamma = 0.1
alpha = 2
beta = 1/16

## Calibrating the functions

In [34]:
test_tester = tester_calibration(gamma, alpha, beta, step_gamma=0.1, step_alpha=0.5, step_beta=0.1, step_tax=0.001)
test_tester

,Gamma,Beta,Alpha,High Endowment Tax,Low Endowment Tax,Tax Difference
0,0.1,0.0625,2.0,0.551,0.551,0.0
1,0.1,0.0625,2.5,0.551,0.551,0.0
2,0.1,0.0625,3.0,0.551,0.551,0.0
3,0.1,0.0625,3.5,0.551,0.551,0.0
4,0.1,0.0625,4.0,0.551,0.551,0.0
...,...,...,...,...,...,...
37235,1.9,0.9625,97.5,0.498,0.498,0.0
37236,1.9,0.9625,98.0,0.498,0.498,0.0
37237,1.9,0.9625,98.5,0.498,0.498,0.0
37238,1.9,0.9625,99.0,0.498,0.498,0.0


In [35]:
test_tester.to_excel("calibration.xlsx")

D:\Anaconda5.2.0\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.9.3' currently installed).
  warnings.warn(msg, UserWarning)
